# Sqlalchemy tutorial
### Python's SQLAlchemy and Object-Relational Mapping

A common task when programming any web service is the construction of a solid database backend. In the past, programmers would write raw SQL statements, pass them to the database engine and parse the returned results as a normal array of records. Nowadays, programmers can write Object-relational mapping (ORM) programs to remove the necessity of writing tedious and error-prone raw SQL statements that are inflexible and hard-to-maintain. This tutorial is provided by [pythoncentral](https://www.pythoncentral.io/introductory-tutorial-python-sqlalchemy/).

<p align="center">
<img src="https://www.pythoncentral.io/wp-content/uploads/2013/04/SQLAlchemyPersonAddress.png" alt="OSI layer model example."/></p>

In [1]:
'''
sqlite_ex.py
'''
import sqlite3

con = sqlite3.connect('Example.db')
cur = con.cursor()

cur.execute('''
            CREATE TABLE person
            (id INEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
            ''')
cur.execute('''
            CREATE TABLE address
            (id INTEGER PRIMARY KEY ASC, street_name varchar(250), street_number varchar(250), post_code varchar(250) NOT NULL, person_id INTEGER NOT NULL, FOREIGN KEY(person_id) REFERENCES person(id))
            ''')

cur.execute("INSERT INTO person VALUES(1, 'pythoncentral')")
cur.execute("INSERT INTO address VALUES(1, 'python road', '1', '00000', 1)")

con.commit()
con.close()

In [10]:
'''
sqlite_query.py

This example shows how to use the CURD (Create, Update, Read, Delete) data in the database.
'''
import sqlite3
con = sqlite3.connect('Example.db')

cur = con.cursor()
cur.execute('SELECT * FROM person')
print(cur.fetchall())
cur.execute('SELECT * FROM address')
print(cur.fetchall()[0])
cur.execute('UPDATE person SET name = ?',('pythonmain',))
cur.execute('SELECT * FROM person')
print(cur.fetchall())
cur.execute("INSERT into person VALUES(2, 'pythondivision')")
cur.execute('SELECT * FROM person')
print(cur.fetchall())
cur.execute('DELETE FROM person WHERE id=?',(1,))
cur.execute('SELECT * FROM person')
print(cur.fetchall())
con.close()

[(1, 'pythoncentral')]
(1, 'python road', '1', '00000', 1)
[(1, 'pythonmain')]
[(1, 'pythonmain'), (2, 'pythondivision')]
[(2, 'pythondivision')]


### Python's SQLAlchemy and Declarative
There are three most important components in writing SQLAlchemy code:

    - A Table that represents a table in a database.
    - A mapper that maps a Python class to a table in a database.
    - A class object that defines how a database record maps to a normal Python object.

Instead of having to write code for Table, mapper and the class object at different places, SQLAlchemy's declarative allows a Table, a mapper and a class object to be defined at once in one class definition.

In [44]:
'''
sqlalchemy_declarative.py

This example shows how to setup a data base with sqlalchemy.
'''
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

Base = declarative_base()

class Person(Base):
    __tablename__= 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250),nullable=False)
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.create_all(engine)

In [45]:
'''
sqlalchemy_insert.py

This example shows how to store data into the sqlalchemy data base.
'''
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#from sqlalchemy_declaritive import Address, Base, Person

engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()

new_person = Person(name='Nawin')
session.add(new_person)
session.commit()

new_address = Address(post_code='48103',street_name ='West Huron Street', person=new_person)
session.add(new_address)
session.commit()


In [63]:
'''
sqlalchemy_declarative.py

This example shows how to query the data from the sqlalchemy data base.
'''

#from sqlalchemy import Person, Base, Address
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///sqlalchemy_example.db')
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()

for i in session.query(Person).all():
    print("Person Id:{} Name:{}".format(i.id, i.name))

person = session.query(Person).first()

print("Person Id:{} Name:{}".format(person.id, person.name))

address = session.query(Address).first()
print("Address Id:{} Person Name: {}".format(address.id, address.person.name))

finding = session.query(Person).filter(Person.name==address.person.name).first()

print("Address Id:{} Person Name: {}".format(finding.id, finding.name))


Person Id:1 Name:Nawin
Person Id:2 Name:Nawin
Person Id:3 Name:Nawin
Person Id:4 Name:Nawin
Person Id:5 Name:Tim
Person Id:6 Name:Tim
Person Id:1 Name:Nawin
Address Id:1 Person Name: Nawin
Address Id:1 Person Name: Nawin


### Summary of Python's SQLAlchemy

In this article, we learned how to write database code using SQLAlchemy's declaratives. Compared to writing the traditional raw SQL statements using sqlite3, SQLAlchemy's code is more object-oriented and easier to read and maintain. In addition, we can easily create, read, update and delete SQLAlchemy objects like they're normal Python objects.

You might be wondering that if SQLAlchemy's just a thin layer of abstraction above the raw SQL statements, then it's not very impressive and you might prefer to writing raw SQL statements instead. In the following articles of this series, we're going to investigate various aspects of SQLAlchemy and compare it against raw SQL statements when they're both used to implement the same functionalities. I believe at the end of this series, you will be convinced that SQLAlchemy is superior to writing raw SQL statements.